# Setup

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# replace the dir if need
DOOR_DIR = "/content/drive/MyDrive/OneCareer/Project - G5/Dataset/" # Henry's dir
# DOOR_DIR = "/content/drive/MyDrive/Project - G5/Dataset/"    #Lejia's dir
#DOOR_DIR = "/content/drive/MyDrive/Project - G5/Dataset/" #Nan's dir
#DOOR_DIR = "/content/drive/MyDrive/Project - G5/Dataset" 

f1 = DOOR_DIR + "MeetFresh_Menu_Drink.csv"
f2 = DOOR_DIR + "MeetFresh_Menu_Food.csv"
f3 = DOOR_DIR + "survey.csv"

In [ ]:
# df1 = pd.read_csv(f1)
df2 = pd.read_csv(f2)
df3 = pd.read_csv(f3)

# EDA

In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   user_id                       92 non-null     int64 
 1   gender                        92 non-null     object
 2   age                           92 non-null     int64 
 3   ethnicity                     92 non-null     object
 4   sensitivity on price          92 non-null     int64 
 5   sensitivity on calorie        92 non-null     int64 
 6   sensitivity on seasonal food  92 non-null     int64 
 7   hot or cold                   90 non-null     object
 8   current state                 92 non-null     object
 9   Favorite Dish                 92 non-null     object
 10  Caffeine                      92 non-null     int64 
 11  Cornstarch                    92 non-null     int64 
 12  Dairy                         92 non-null     int64 
 13  Gluten                

In [ ]:
df3.head(3)

,user_id,gender,age,ethnicity,sensitivity on price,sensitivity on calorie,sensitivity on seasonal food,hot or cold,current state,Favorite Dish,Caffeine,Cornstarch,Dairy,Gluten,Mango,Nuts,Peanuts,Soybean,Tapioca Starch,Egg
0,1,F,23,Asian,1,3,3,C,CT,C1,0,0,0,0,0,1,0,0,0,0
1,2,F,24,Asian,5,5,3,C,CT,A1,0,0,0,0,0,0,0,0,0,0
2,3,F,18,Asian,2,4,1,C,DC,A2,0,0,0,0,0,0,0,0,0,0


In [ ]:
df3['sensitivity on price'].unique()

array([1, 5, 2, 3, 4])

In [ ]:
#sensitive plot
import plotly.graph_objs as go
go.Figure(
    data=[go.Histogram(x=df3["sensitivity on price"], xbins={"start": 1.0, "end": 5.2, "size": 0.25})],
    layout=go.Layout(title="Histogram of sensitivity on price", yaxis={"title": "Count"}, bargap=0.05),
    )

In [ ]:
# Age histogram
go.Figure(
    data=[go.Histogram(x=df3["age "], xbins={"start": 15, "end": 66, "size": 3.0})],
    layout=go.Layout(title="Histogram of age ", yaxis={"title": "Count"}, bargap=0.05),
    )

In [ ]:
# Peanut 
import plotly.express as px
fig = px.violin(df3, x='Peanuts')
fig

# Preprocessing

In [ ]:
df3['gender'].replace(['F', 'M'], [0, 1], inplace=True)
df3['ethnicity'].replace(['Asian', 'White','Black','Hispanic'], [1,2,3,4], inplace=True)
df3['hot or cold'].replace(['H', 'C'], [0,1], inplace=True)
df3.head()

,user_id,gender,age,ethnicity,sensitivity on price,sensitivity on calorie,sensitivity on seasonal food,hot or cold,current state,Favorite Dish,Caffeine,Cornstarch,Dairy,Gluten,Mango,Nuts,Peanuts,Soybean,Tapioca Starch,Egg
0,1,0,23,1,1,3,3,1.0,CT,C1,0,0,0,0,0,1,0,0,0,0
1,2,0,24,1,5,5,3,1.0,CT,A1,0,0,0,0,0,0,0,0,0,0
2,3,0,18,1,2,4,1,1.0,DC,A2,0,0,0,0,0,0,0,0,0,0
3,4,1,15,1,3,1,2,1.0,DC,D11,0,0,0,0,0,0,0,0,0,0
4,5,0,18,1,4,4,3,1.0,CT,C5,0,0,0,0,0,0,0,0,0,0


# Model

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
new_description = pd.Series('F Asian 2')

## Sample

In [ ]:
v1 = np.array([[0, 0, 2]])
v2 = np.array([[1, 0, 5]])

print(cosine_similarity(v2, v1))

[[0.98058068]]


## Actual Model

In [ ]:
# drop unused col
# Note: this step is needed since we don't ask these three features from the user's input
df3_drop = df3.drop(['current state','Favorite Dish ','user_id'], axis=1)

In [ ]:
raw_input = [0, 24, 1, 5, 5, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
input = np.array([raw_input], dtype=np.int32)

def compute_cosine(input: np.array, df: pd.DataFrame) -> list():
  '''
    Compute recommendation based on one user profile (aka user feature vector) 
    against with all other users in the sample.

    INPUT:
      input: 2D numpy array with 17 user features
      df: pandas dataframe type which contains n user survey data

    OUTPUT:
      result: a list of cosine similarity score
        - note: the order of the list is in the correct corresponding the row of the sample,
                any col shuffle must be extended across the sample.
  '''
  result = list()

  for i in range(len(df.index)):
    result.append(cosine_similarity(
        input,
        np.array([df.iloc[i].values], dtype=np.int32)
        )
    )
  
  return result

In [ ]:
sim_list = compute_cosine(input, df3_drop)

In [ ]:
sim_list

---

In [ ]:
from pandas.util.testing import assert_frame_equal

class product_return(object): 
  '''
  Return recommended product based on similarity between users who access the content 
  '''
  def __init__(self,match_list,product_data,customer_data):
    self.sim_list = sim_list 
    self.product_data = df2
    self.customer_data = df3
    self.T=0.99
  
  def product_recommended_code(self):
    self.customer_data['rank_score'] = self.sim_list
    self.customer_data['rank_score'] = self.customer_data['rank_score'].astype(float, errors = 'raise')
    most_similar_user = self.customer_data[self.customer_data['rank_score']>self.T].sort_values('rank_score').head(5)
    ##most_similar_user = pd.DataFrame(most_similar_user).transpose()
    product_recommended_code = most_similar_user['Favorite Dish ']
    return product_recommended_code


In [ ]:
product_recommended_code = product_return(sim_list,df2,df3).product_recommended_code()
print(product_recommended_code)

84     A1
34     A3
42     C2
5     D13
26     B2
Name: Favorite Dish , dtype: object


In [ ]:
product_recommended_code = product_recommended_code[1]

In [ ]:
df2.head()

,Item ID,Menu Category,Item Name (CHN),Item Name (ENG),Hot,Cold,Size,Kcal,Egg Allergen,Peanut Allergen,...,Strawberry,Mango,Egg Waffle,Matcha Egg Waffle,Chocolate Egg Waffle,Chocolate Chips,Matcha Red Bean,Mixed Nuts,Chocolate Wafer Rolls,Chocolate Syrup
0,A1,Signature Series,芋圆招牌,Icy Taro Ball Signature,0.0,1.0,NaN,694,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A2,Signature Series,鲜芋仙招牌,Icy Grass Jelly Signature,0.0,1.0,NaN,420,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A3,Signature Series,双芋招牌,Double Taro Signature,0.0,1.0,NaN,586,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A4,Signature Series,招牌红豆汤,Hot Red Bean Soup Signature,1.0,0.0,NaN,1135,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A5,Signature Series,招牌烧仙草,Hot Grass Jelly Soup Signature,1.0,0.0,NaN,944,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
item_name = df2.loc[df2['Item ID'] == product_recommended_code, 'Item Name (ENG)']

In [ ]:
item_name

0    Icy Taro Ball Signature
Name: Item Name (ENG), dtype: object

##Web Application

In [ ]:
!pip install -q streamlit

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2022-05-09 21:43:05--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  6.07MB/s    in 2.2s    

2022-05-09 21:43:08 (6.07 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [ ]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://8f58-35-221-181-144.ngrok.io


2022-05-09 21:43:58.862 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.221.181.144:8501

  Stopping...
  Stopping...


In [ ]:
import streamlit as st
st.title("Hello GeeksForGeeks !!!")

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [ ]:
!streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py

2022-05-09 21:50:39.887 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.221.181.144:8501

  Stopping...
  Stopping...
